In [36]:
from pprint import pprint
import draco as drc

import altair as alt
from draco.renderer import AltairRenderer
from vega_datasets import data

d = drc.Draco()
renderer = AltairRenderer()
df = data.cars()


def change_column_name(df):
  df.columns = df.columns.str.lower()
  return df


df = change_column_name(df)
base_scheme = drc.schema_from_dataframe(df)
base_facts = drc.dict_to_facts(base_scheme)
# base_facts = [
#   *base_facts,
#   "entity(view,root,v0).",
#   "entity(mark,v0,m).",
#   "entity(encoding,m,e0).",
#   "attribute((encoding,field),e0,year).",
#   "entity(encoding,m,e1).",
#   "attribute((encoding,field),e1,horsepower).",
# ]

# model = next(draco.complete_spec(base_facts, 10))
# spec = answer_set_to_dict(model.answer_set)
# pprint(model)
# display(renderer.render(spec, df))

# @app.get("/")


In [56]:
df.head()

,name,miles_per_gallon,cylinders,displacement,horsepower,weight_in_lbs,acceleration,year,origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


In [39]:
def recommend_charts(
  spec: list[str],
  draco: drc.Draco,
  num: int = 5,
  labeler=lambda i: f"CHART {i+1}",
) -> dict[str, tuple[list[str], dict]]:
  # Dictionary to store the generated recommendations, keyed by chart name
  chart_specs = {}
  for i, model in enumerate(draco.complete_spec(spec, num)):
    chart_name = labeler(i)
    spec = drc.answer_set_to_dict(model.answer_set)
    chart_specs[chart_name] = drc.dict_to_facts(spec), spec

    print(chart_name)
    print(f"COST: {model.cost}")
    chart = renderer.render(spec=spec, data=df)
    # Adjust column-faceted chart size
    if (
      isinstance(chart, alt.FacetChart)
      and chart.facet.column is not alt.Undefined
    ):
      chart = chart.configure_view(continuousWidth=130, continuousHeight=130)
    display(chart)

  return chart_specs


In [65]:
pprint(base_facts)

['attribute(number_rows,root,406).',
 'entity(field,root,0).',
 'attribute((field,name),0,name).',
 'attribute((field,type),0,string).',
 'attribute((field,unique),0,311).',
 'attribute((field,entropy),0,5617).',
 'attribute((field,freq),0,6).',
 'entity(field,root,1).',
 'attribute((field,name),1,miles_per_gallon).',
 'attribute((field,type),1,number).',
 'attribute((field,unique),1,129).',
 'attribute((field,entropy),1,4312).',
 'attribute((field,min),1,9).',
 'attribute((field,max),1,46).',
 'attribute((field,std),1,7).',
 'entity(field,root,2).',
 'attribute((field,name),2,cylinders).',
 'attribute((field,type),2,number).',
 'attribute((field,unique),2,5).',
 'attribute((field,entropy),2,1103).',
 'attribute((field,min),2,3).',
 'attribute((field,max),2,8).',
 'attribute((field,std),2,1).',
 'entity(field,root,3).',
 'attribute((field,name),3,displacement).',
 'attribute((field,type),3,number).',
 'attribute((field,unique),3,83).',
 'attribute((field,entropy),3,3982).',
 'attribute

In [90]:
r = recommend_charts(
  spec=[
    *base_facts,
    "entity(view,root,v0).",
    "entity(mark,v0,m).",
    "entity(encoding,m,e0).",
    "attribute((encoding,field),e0,weight_in_lbs).",
    "attribute((encoding,channel),e0,x).",
    # "attribute((encoding,binning),e0, 10).",
    "entity(encoding,m,e1).",
    "attribute((encoding,aggregate),e1,count).",
    # "entity(encoding,m,e2).",
    # "attribute((encoding,field),e2,displacement).",
    # "entity(encoding,m,e1).",
    # "attribute((encoding,field),e1,origin).",
    ":- {entity(encoding,_,_)} > 3.",
    # ":- {entity(encoding,_,_)} < 3.",
    ":- {entity(mark,_,_)} != 1.",
  ],
  draco=d,
)

CHART 1
COST: [9]


alt.Chart(...)

CHART 2
COST: [9]


alt.Chart(...)

CHART 3
COST: [10]


alt.Chart(...)

CHART 4
COST: [10]


alt.Chart(...)

CHART 5
COST: [11]


alt.Chart(...)